# Most Popular model

This is base on this [notebook](https://www.kaggle.com/dvasyukova/expedia-hotel-recommendations/predict-hotel-type-with-pandas) by dune_dweller.

Submission scored 0.30253. 

This was run on an AWS ECS r3.8xlarge (32 cores, 240GB memory cost 5c/hr), Total run time of about 78s, with about 70s being reading and writting the CSVs

## Initalise
set page width and load graphlab etc

In [3]:
# Set full page width
from IPython.core.display import HTML
HTML("""
<style>
.container {
    width: 100%;
}
</style>
""")

In [4]:
import graphlab as gl
gl.canvas.set_target('ipynb')
import datetime

## Create a summary

Sumamrise by srch_destination_id, tallying the bookings and click and creating a relevance score. 

In [5]:
train = gl.SFrame('Data/train.csv')
print "Train:", len(train)

2016-05-04 01:37:56,761 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.9 started. Logging: /tmp/graphlab_server_1462325875.log


Finished parsing file /home/ec2-user/Notebooks/ExpediaHotelRecommendations/Data/train.csv

Parsing completed. Parsed 100 lines in 1.2729 secs.

This non-commercial license of GraphLab Create is assigned to kevin.mcisaac@gmail.com and will expire on November 06, 2016. For commercial licensing options, visit https://dato.com/buy/.
------------------------------------------------------

Read 485475 lines. Lines per second: 237111

...

Read 31540717 lines. Lines per second: 800566

Read 35420132 lines. Lines per second: 797783

Finished parsing file /home/ec2-user/Notebooks/ExpediaHotelRecommendations/Data/train.csv

Parsing completed. Parsed 37670293 lines in 47.1449 secs.


Inferred types from first 100 line(s) of file as 
column_type_hints=[str,int,int,int,int,int,float,int,int,int,int,str,str,int,int,int,int,int,int,int,int,int,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
Train: 37670293


The CSV was imported in 53s 

In [6]:
summary = train.groupby(['srch_destination_id', 'hotel_cluster'], 
                        {'bookings':gl.aggregate.SUM('is_booking'), 
                         'clicks': gl.aggregate.COUNT()} )
summary

hotel_cluster,srch_destination_id,bookings,clicks
35,20713,1,8
81,22928,0,1
61,37232,0,1
91,49580,1,13
94,12254,2,31
82,21027,14,113
3,12875,0,1
47,12972,1,19
32,42294,0,7
14,12592,2,8


In [7]:
BOOKING_WEIGHT = 100
summary['clicks'] = summary['clicks'] - summary['bookings'] 
summary['relevance'] = summary['bookings']*BOOKING_WEIGHT + summary['clicks']
summary

hotel_cluster,srch_destination_id,bookings,clicks,relevance
35,20713,1,7,107
81,22928,0,1,1
61,37232,0,1,1
91,49580,1,12,112
94,12254,2,29,229
82,21027,14,99,1499
3,12875,0,1,1
47,12972,1,18,118
32,42294,0,7,7
14,12592,2,6,206


# Find most popular hotel clusters by destination
Define a function to get most popular hotels for a destination group.
nlargest() function lets us save some time on sorting large groups.


In [8]:
def arg_max(d, k=3):
    topk = sorted(d.keys(), reverse=True)[:k]
    
    return  [str(d[k]) for k in topk]

In [9]:
most_popular = summary.groupby('srch_destination_id',
                       {'hotel_cluster': gl.aggregate.CONCAT('relevance', 'hotel_cluster')})

most_popular['hotel_cluster'] = most_popular['hotel_cluster'].apply(lambda d: arg_max(d, k=5), list)
most_popular.print_rows(5, max_column_width=40)

Using default 16 lambda workers.

To maximize the degree of parallelism, add the following code to the beginning of the program:

"graphlab.set_runtime_config('GRAPHLAB_DEFAULT_NUM_PYLAMBDA_WORKERS', 32)"

Note that increasing the degree of parallelism also increases the memory footprint.

+---------------------+----------------------+
| srch_destination_id |    hotel_cluster     |
+---------------------+----------------------+
|        26439        | [91, 48, 40, 14, 39] |
|        22258        |  [99, 9, 46, 2, 68]  |
|        26175        | [10, 43, 50, 3, 61]  |
|        11106        |         [81]         |
|         9234        |       [23, 93]       |
+---------------------+----------------------+
[59455 rows x 2 columns]



In [10]:
test=gl.SFrame('Data/test.csv')

Finished parsing file /home/ec2-user/Notebooks/ExpediaHotelRecommendations/Data/test.csv

Parsing completed. Parsed 100 lines in 0.43092 secs.

Read 483190 lines. Lines per second: 676280

Finished parsing file /home/ec2-user/Notebooks/ExpediaHotelRecommendations/Data/test.csv

Parsing completed. Parsed 2528243 lines in 2.7737 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,int,int,int,int,int,float,int,int,int,int,str,str,int,int,int,int,int,int,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


In [11]:
test = test.join(most_popular, how='left',on='srch_destination_id')
test

id,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city
0,2015-09-03 17:09:54,2,3,66,174,37449
1,2015-09-24 17:38:35,2,3,66,174,37449
2,2015-06-07 15:53:02,2,3,66,142,17440
3,2015-09-14 14:49:10,2,3,66,258,34156
4,2015-07-17 09:32:04,2,3,66,467,36345
5,2015-07-21 11:58:45,2,3,66,311,48189
6,2015-07-29 07:58:39,2,3,66,311,48189
7,2015-08-01 20:13:15,2,3,66,348,24811
8,2015-11-07 12:29:09,2,3,66,311,48189
9,2015-11-08 16:21:37,2,3,66,311,48189


In [12]:
test['hotel_cluster'].num_missing()

14036

In [13]:
most_popular_all = summary.groupby('hotel_cluster',
                       {'relevance': gl.aggregate.SUM('relevance')})

most_popular_all = list(most_popular_all.topk('relevance', k=5)['hotel_cluster'].astype(str))
most_popular_all

['91', '48', '42', '59', '28']

In [14]:
test = test.fillna('hotel_cluster', most_popular_all)

In [15]:
test['hotel_cluster'] = test['hotel_cluster'].apply(lambda lst: ' '.join(lst))

In [16]:
test['srch_destination_id', 'hotel_cluster']

srch_destination_id,hotel_cluster
12243,55 37 5 22 11
14474,5
11353,0 31 77 91 59
8250,1 45 79 24 54
11812,91 2 42 59 48
11827,91 42 28 95 4
8271,95 2 21 98 33
8291,95 18 98 91 68
8250,1 45 79 24 54
9145,55 32 34 10 4


write out submission

In [17]:
test['id', 'hotel_cluster'].save('Submissions/MostPopular.csv')
# submission scored 0.30253